# Simple Example of using WebLoader with Keras

In [1]:
#!test -f training.tgz || curl http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_train-0000-000.tgz -o training.tgz
#!test -f testing.tgz || curl http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000.tgz -o testing.tgz

In [3]:
from importlib import reload
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import models, layers
import numpy as np

from keras.utils.training_utils import multi_gpu_model
import os
import numpy as np
import webloader as wl

from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [5]:
#!test -f imagenet_val-0000-000.tgz || curl http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz -o imagenet_val-0000.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 16 5575M   16  922M    0     0  54.2M      0  0:01:42  0:00:17  0:01:25 60.0M^C


In [13]:
num_gpus = 1

urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_train-{0000..0147}-{000..019}.tgz"
#urls = "training.tgz"
training_size = 1000000
batch_size = 64
num_batches = training_size//batch_size


if os.path.exists("imagenet_val-0000.tgz"):
    test_urls = "imagenet_val-0000.tgz"
else:
    test_urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz"
print("using", test_urls)
test_size = 50000
test_batch_size = 256
test_num_batches = test_size//test_batch_size

using http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz


In [14]:
mean = np.array([0.485, 0.456, 0.406], "f")
std = np.array([0.229, 0.224, 0.225], "f")
def norm_image(xs):
    return (xs-mean[None,None,None,:])/std[None,None,None,:]
def norm_cls(ys):
    return keras.utils.to_categorical(ys-1, 1000)

In [15]:
testing = wl.WebLoader(test_urls, test_size,
                       fields="ppm;png;jpg cls",
                       shuffle=1000,
                       batch_transforms=[norm_image, norm_cls],
                       batch_size=test_batch_size,
                       epochs=-1)
wl.loader_test(testing)

294.22 samples/s 1.15 batches/s
0 : ndarray (256, 224, 224, 3) float32 -2.117904 2.64
1 : ndarray (256, 1000) float32 0.0 1.0


In [16]:
training = wl.MultiWebLoader(urls, training_size,
                       fields="ppm;png;jpg cls",
                       batch_transforms=[norm_image, norm_cls],
                       batch_size=batch_size,
                       processes=4)
wl.loader_test(training)

744.91 samples/s 11.64 batches/s
0 : ndarray (64, 224, 224, 3) float32 -2.117904 2.64
1 : ndarray (64, 1000) float32 0.0 1.0


In [17]:
lr = 0.001 # was: 0.0001

model = ResNet50()

if num_gpus>1:
    model = multi_gpu_model(model, gpus=num_gpus)

opt = keras.optimizers.rmsprop(lr=lr, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit_generator(iter(training),
              epochs=100,
              steps_per_epoch=num_batches, #num_batches,
              validation_data=iter(testing),
              validation_steps=test_num_batches, #test_num_batches,
              shuffle=True,
              use_multiprocessing=True,
              workers=1)

W0625 16:47:25.090767 140175026829120 deprecation_wrapper.py:119] From /home/tmb/exp/webloader/venv/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0625 16:47:25.199564 140175026829120 deprecation.py:323] From /home/tmb/exp/webloader/venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
   12/15625 [..............................] - ETA: 64:22:05 - loss: 8.1690 - acc: 0.0521

Process ForkPoolWorker-10:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/u

KeyboardInterrupt: 